# **Analysis of Movie Industry ROIs**
## Malcolm Katzenbach, Lauren Phipps, Dan Valenzuela

***



In [1]:
import CustomLibrary as cl

cl.import_libs()
%matplotlib inline

## **Overview** <a id="Overview"></a>

[**1. Business Problem**](#Business-Problem)

[**2. Data Understanding**](#Data-Understanding)

[**3. Data Preparation**](#Data-Preparation)

[**4. Data Analysis**](#Data-Analysis)

[**5. Evaluation**](#Evaluation)

[**6. Conclusion**](#Conclusion)
***

## **Business Problem** <a id="Business-Understanding"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Data Understanding** <a id="Data-Understanding"></a>
[*↑ Back to overview*](#Overview)

### Datasets
For the purposes of this analysis we focused primarily on data from the Internet Movie Database (IMDB) and The-Numbers.com. Specifically we used datasets that included--on one hand--title, date released, and genre data and--on the other hand--title, date released, production budgets, and box office figures. Below is a summary of the data pertinent to our analysis broken down by file. 

| imdb.title.basics.csv | tn.movie_budgets.csv |
| --- | --- |
| primary_title | movie |
| start_year | release_date |
| genres |  |
|  | production_budget |
|  | domestic_gross |
|  | worldwide_gross |


In this table you can see our understanding of how data between the two datasets can "match" in the sense that they provide they same kind of data but may be in different formats. For example, `start_year` in `imdb.title.basics.csv` is  formatted as `YYYY` whereas `release_date` in `tn.movie_budgets.csv` is formatted as `MMM DD, YYYY`. Further, they may be stored as different data types. Release date data across both sets are in 

[*↑ Back to overview*](#Overview)
***

## **Data Preparation** <a id="Data-Preparation"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Data Analysis** <a id="Data-Analysis"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Evaluation**<a id="Evaluation"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Conlcusion**<a id="Conclusion"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***